In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

pyro.set_rng_seed(101)

In [2]:
def scale(guess):
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    return pyro.sample("measurement", dist.Normal(weight, 0.75))

# 条件付け

* 確率的プログラミングの真の力は、生成モデルを観測値で条件付け、そのデータを生成しうる潜在変数の推論するという能力にある
* Pyro の API には、生成モデルの定義と条件付けを分離させることで、何回でもデータに対して条件付ける操作を行えるような工夫が施されている

`scale` をもう一度考える

* `guess = 8.5` に対する `weight` の分布は、`measurement == 9.5` が観測されていた場合どうなるだろうか？
* つまり次の分布を推論したい

$$
P(weight|guess, measurement=9.5)
$$

* Pyro では `pyro.condition` を使ってデータの条件付けを行う
* `pyro.condition` は高階関数
    * モデルと観測値を入力する
    * 出力はモデルと一緒の外見だが、データで条件付けられたモデル

In [4]:
conditioned_scale = pyro.condition(scale, data={'measurement': 9.5})

任意の Python の関数のように振る舞うので、`lambda` や `def` のように遅延させることも出来ます。

In [5]:
def deffered_conditioned_scale(measurement, *args, **kwargs):
    return pyro.condition(scale, data={'measurement': measurement})(*args, **kwargs)

条件付けをモデル定義に盛り込むこともできる

* `pyro.sample` のキーワード引数として `obs` を追加すれば良い
* この操作は後で `pyro.condition` するのと等価

In [6]:
def scale_obs(guess):
    # conditioned_scale と等価
    weight = pyro.sample("weight", dist.Normal(guess, 1.))
    return pyro.sample("measurement", dist.Normal(weight, 1.), obs=9.5)

観測値をモデルに組み込む違う方法として、`pyro.do` というものがある。
- これは、Pearl の do-operator の実装であり、因果モデルの観測値に対応する。
- `pyro.do` と `pyro.condition` は自由にミックスさせて使うことが出来る。